In [0]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix 
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics  import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
# import data

#data_arrv = pd.read_csv("CHO_arrival_v3.csv")
#data_weath = pd.read_csv("CHO_weather.csv")

In [0]:
#upload files to work here
# (Pat) I used CHO_arrival_v3 and this file is the latest one I worked for adding some features I think it is useful and set up for the tuning parameters 
# so that it will be faster when all data are ready
from google.colab import files
uploaded = files.upload()

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
data_arrv = pd.read_csv("CHO_arrival_v3.csv")
data_weath = pd.read_csv("CHO_weather.csv")

In [0]:
data_weath.head()

**Preprocessing Data on the Weather Part**

In [0]:

# Choose only the daily data
data_weath = data_weath.loc[data_weath['REPORT_TYPE'] == 'SOD  ']

In [0]:
print(type(data_weath))
print(type(data_weath['HourlyDewPointTemperature']))
H_dewPointT = data_weath['HourlyDewPointTemperature']

In [0]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [0]:
average_day = []
avg_value = 0
count = 0
#print(type(H_dewPointT[1]))
for dp in H_dewPointT:
    if isinstance(dp, str): 
        if isfloat(dp):
            avg_value += float(dp)
            count += 1
    else:
        if count != 0:
            day_avg = avg_value / count 
            average_day.append(day_avg)
            avg_value = 0
            count = 0
print((average_day))

In [0]:
# NEED TO FIX HERE
# Try to add more features by averaging the hourly data to represent daily data
# # Make this part to find missing daily info by averaging hourly data
# data_all = data_weath.loc[data_weath['REPORT_TYPE'] != 'SOM  ']
# #data_all = data_weath.loc[data_weath['REPORT_TYPE'] != 'SOD  ']

# # DailyAverageDewPointTemperature
# index_daily = list(data_weath.index)
# average = []
# for i in range(0, len(index_daily)):
#     summ = 0;
#     if i == 0:
#         n = index_daily[i]
#         for j in range(0, index_daily[i]):
#             #print((data_all.loc[data_all.index[j], 'HourlyDewPointTemperature']))
#             summ = summ + int(data_all.loc[data_all.index[j], 'HourlyDewPointTemperature'])
#         average.append(float(summ/n))
#     else:
        
#         n = index_daily[i] - index_daily[i-1]
#         #print(data_all.loc[data_all.index[0], 'HourlyDewPointTemperature'])
#         for j in range(index_daily[i-1]+1, index_daily[i]):
#             #print(str(j) + " " + data_all.loc[data_all.index[j], 'HourlyDewPointTemperature'])
#             summ = summ + int(data_all.loc[data_all.index[j], 'HourlyDewPointTemperature'])
#         print(float(summ/n))
#         average.append(float(summ/n))
    
# #     #print(data_weath.loc[data_weath.index[i], 'DailyWeather'])
# #     if data_weath.loc[data_weath.index[i], 'DailyWeather'] == '':
# #         data_weath.loc[data_weath.index[i], 'DailyWeather'] = data_weath.loc[data_weath.index[i-1], 'DailyWeather']
# print(average)

In [0]:
# Drop unrelated features
data_weath = data_weath.drop(["AWND", "BackupDirection", "BackupDistance", "BackupDistanceUnit", "BackupElements", "BackupElements", "BackupElevation","BackupElevationUnit", "BackupEquipment", "BackupLatitude", "BackupLongitude", "BackupName", "CDSD", "CLDD", "DSNW"], axis=1)

In [0]:
# Continue dropping
list_drop = []
for i in range(24,110):
    list_drop.append(i)
#print(len(list_drop))
#print(list_drop)
data_weath = data_weath.drop(data_weath.columns[list_drop], axis=1)
data_weath = data_weath.drop(["STATION", "REPORT_TYPE", "SOURCE", "DailySnowDepth", "DailySnowfall"], axis = 1)

# Will fix this later (can make it more accurate data when having these features from hourly data)
data_weath = data_weath.drop(["DailyAverageDewPointTemperature", "DailyAverageRelativeHumidity", "DailyAverageSeaLevelPressure", 
"DailyAverageWetBulbTemperature"], axis = 1)


In [0]:
data_weath.columns

In [0]:
# trace amount = 0
data_weath = data_weath.replace('T', 0)
data_weath = data_weath.replace('Ts', 0)

# Anything has 's'
data_weath = data_weath.replace(to_replace=r's$', value='', regex=True)

# Change the format of the date to easier to track
data_weath = data_weath.replace(to_replace=r'T23:59:00$', value='', regex=True)

#print(len(data_weath))
# for i in range(0, len(data_weath)):
#     #print(data_weath.loc[data_weath.index[i], 'DailyWeather'])
#     if data_weath.loc[data_weath.index[i], 'DailyWeather'] == '':
#         data_weath.loc[data_weath.index[i], 'DailyWeather'] = data_weath.loc[data_weath.index[i-1], 'DailyWeather']

#data_weath.index[0]

# fill data with * or NaN with previous value
data_weath = data_weath.replace(to_replace=r'\*', value=np.nan, regex=True)

# fill data with previous value for the Daily Weather
data_weath = data_weath.fillna(method='ffill')


In [0]:
# Try to find missing value in data_weath
# The missing value is 2012-02-28
# Decide to delete the data on data_arr to match the same number in the other data set
count = 0
# index_daily = list(data_weath.index)
# for i in range(0, len(index_daily)):
#     if data_weath.loc[data_weath.index[i], "DATE"].str.contains('2010', regex=True):
#         count = count + 1
year = data_weath["DATE"].str.contains('2012-02', regex=True)
year.value_counts()
# year_arrv = data_arrv["Time"].str.contains('^2010', regex = true)
# year_arrv.value_counts()
#data_weath["DATE"]
#data_arrv["Time"]
#data_arrv["Time"].str.contains('^2010', regex = True)


In [0]:
# Reverse the arrival data to match with the format in data_weather
data_arrv = data_arrv.iloc[::-1]

In [0]:
date = []
for i in range(1,366)
    data.append(i)

In [0]:
# Need to combine both data to further step
# The data is not combined well because of different indexes
# change indexes of both datasets to be the same
index_weath = data_weath.index

data_arrv = data_arrv.set_index(index_weath)
data_weath['DayInYear'] = data_arrv['DayInYear']
data_weath['ArrivalRate'] = data_arrv['Airline On-Time Arrival Rate CHO Charlottesville VA: Charlottesville Albemarle  - %']
data_weath.head()

In [0]:
data_weath.head()

In [0]:
data_weath.to_csv('DailyWeather.csv')

**Plot Correlation Matrix**

In [0]:
# Convert data into float
attributes = ['DailyAverageDryBulbTemperature', 'DailyAverageStationPressure',
       'DailyAverageWindSpeed', 'DailyCoolingDegreeDays',
       'DailyDepartureFromNormalAverageTemperature', 'DailyHeatingDegreeDays',
       'DailyMaximumDryBulbTemperature', 'DailyMinimumDryBulbTemperature',
       'DailyPeakWindDirection', 'DailyPeakWindSpeed', 'DailyPrecipitation',
       'DailySustainedWindDirection', 'DailySustainedWindSpeed', 'ArrivalRate']
for i in range(0, len(attributes)):
    data_weath[attributes[i]] = data_weath[attributes[i]].astype(float)
#data_weath['DailyAverageDryBulbTemperature'] = data_weath['DailyAverageDryBulbTemperature'].astype(int)
#print(type(data_weath.loc[26,'DailyAverageDryBulbTemperature']))

In [0]:
from pandas.plotting import scatter_matrix

scatter_matrix(data_weath[attributes])

In [0]:
# Problem? - The low correlation?
# Add feature?
corr_matrix = data_weath.corr()
corr_matrix["ArrivalRate"].sort_values(ascending=False)

**Pre-Processing on the categorical data**

In [0]:
data_weath_cat = data_weath[['DailyWeather']]
cat_encoder = OneHotEncoder()
data_weath_cat_1hot = cat_encoder.fit_transform(data_weath_cat)
cat_encoder.categories_

In [0]:
len(cat_encoder.categories_[0])

In [0]:
data_weath['DailyWeather'] = data_weath_cat_1hot
data_weath.head()

**Split training and test sets**

In [0]:
# Divide by 1.5 to limit the number of income categories
print(type(data_weath))
data_weath["rate_cat"] = np.ceil(data_weath["ArrivalRate"] / 35)
# Label those above 5 as 5
data_weath["rate_cat"].where(data_weath["rate_cat"] < 5, 5.0, inplace=True)
data_weath["rate_cat"].hist()
print(data_weath["rate_cat"].value_counts())

In [0]:
train_set, test_set = train_test_split(data_weath, test_size=0.3, random_state=42)
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

# FIX HERE
# Plan to manually stratify data instead of using this package
# use stratifiled process to make sure that both train set and test set has the same proportion of negative to positive data
for train_index, test_index in split.split(data_weath, data_weath["rate_cat"]):
    strat_train_set = data_weath.loc[train_index]
    strat_test_set = data_weath.loc[test_index]

In [0]:
strat_train_set.head()

In [0]:
train_set.head()

In [0]:
x_train =  train_set.drop("ArrivalRate", axis = 1)
x_test = test_set.drop("ArrivalRate", axis = 1)
y_train = train_set["ArrivalRate"].copy()
y_test = test_set["ArrivalRate"].copy()

**Pipeline: Combine both numerical and categorical data**

In [0]:
from sklearn.compose import ColumnTransformer

num_attribs = ['DailyAverageDryBulbTemperature', 'DailyAverageStationPressure',
       'DailyAverageWindSpeed', 'DailyCoolingDegreeDays',
       'DailyDepartureFromNormalAverageTemperature', 'DailyHeatingDegreeDays',
       'DailyMaximumDryBulbTemperature', 'DailyMinimumDryBulbTemperature',
       'DailyPeakWindDirection', 'DailyPeakWindSpeed', 'DailyPrecipitation',
       'DailySustainedWindDirection', 'DailySustainedWindSpeed']
cat_attribs = ["DailyWeather"]

full_pipeline = ColumnTransformer([
        ('std_scaler', StandardScaler(), num_attribs),
    ])

x_train = full_pipeline.fit_transform(x_train)
x_test = full_pipeline.fit_transform(x_test)


In [0]:
print(x_train.shape)
print(x_test.shape)

**Training Process**

Linear Regression

In [0]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

scores = cross_val_score(lin_reg, x_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-scores)


def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [0]:
# Train Set
scores = cross_val_score(lin_reg, x_train, y_train,
                         scoring="neg_mean_squared_error", cv=5)
tree_rmse_scores = np.sqrt(-scores)
display_scores(tree_rmse_scores)

In [0]:
# Test Set
scores = cross_val_score(lin_reg, x_test, y_test,
                         scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-scores)
display_scores(lin_rmse_scores)

Tree Regressor

In [0]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(x_train, y_train)

In [0]:
# Train Set
scores = cross_val_score(tree_reg, x_train, y_train,
                         scoring="neg_mean_squared_error", cv=5)
tree_rmse_scores = np.sqrt(-scores)
display_scores(tree_rmse_scores)

In [0]:
# Test Set
scores = cross_val_score(tree_reg, x_test, y_test,
                         scoring="neg_mean_squared_error", cv=5)
tree_rmse_scores = np.sqrt(-scores)

In [0]:
display_scores(tree_rmse_scores)

Forest Regressor

In [0]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=20, random_state=42)
forest_reg.fit(x_train, y_train)

In [0]:
# Train Set
scores = cross_val_score(forest_reg, x_train, y_train,
                         scoring="neg_mean_squared_error", cv=5)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

In [0]:
# Test Set
scores = cross_val_score(forest_reg, x_test, y_test,
                         scoring="neg_mean_squared_error", cv=5)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

**Fine Tuning**

Linear Regression

In [0]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    # try 8 (2×2x2) combinations of hyperparameters
   {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
]

lin_grid_search = GridSearchCV(lin_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
lin_grid_search.fit(x_train, y_train)

In [0]:
best_lin_reg = lin_grid_search.best_estimator_

In [0]:
best_lin_reg

In [0]:
# Train Set
scores = cross_val_score(best_lin_reg, x_train, y_train,
                         scoring="neg_mean_squared_error", cv=5)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

In [0]:
# Test Set
scores = cross_val_score(best_lin_reg, x_test, y_test,
                         scoring="neg_mean_squared_error", cv=5)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

Forest Regressor

In [0]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 50], 'max_features': [6, 8, 10, 11, 12]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10, 50], 'max_features': [4, 6, 8, 10, 12]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(x_train, y_train)

In [0]:
best_forest_reg = grid_search.best_estimator_

In [0]:
best_forest_reg

In [0]:
# Train Set
scores = cross_val_score(best_forest_reg, x_train, y_train,
                         scoring="neg_mean_squared_error", cv=5)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

In [0]:
# Test Set
scores = cross_val_score(best_forest_reg, x_test, y_test,
                         scoring="neg_mean_squared_error", cv=5)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

In [0]:
# # Ignore it, scratch paper
# list_impute = []
# for i in range(8,len(data_weath.columns)-1):
#     list_impute.append(i)
# #imputer = Imputer(strategy="mean", missing_values = '')
# #imputer.fit_transform(data_weath.columns[list_impute])